In [1]:
from utilities import init_bigquery_client
from google.cloud import bigquery
import os
import pandas as pd
import numpy as np
import plotly.express as px

#init BigQuery client
bq = init_bigquery_client()

Using BigQuery credentials: etl-testing-478716-c0b6c2c512e0.json


## Perform bulk queries

In [2]:
# Read from the 'events' table in BigQuery
query = """
    SELECT *
    FROM `etl-testing-478716.firebase_etl_prod.events`
"""
events_df = bq.query(query).to_dataframe()

# Read from the 'userinvites' table in BigQuery
query = """
    SELECT *
    FROM `etl-testing-478716.firebase_etl_prod.userinvites`
"""
userinvites_df = bq.query(query).to_dataframe()

/opt/miniconda3/envs/heyyall/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


## Number of Events Created Per Day with Invites

In [3]:
#filtering out duplicate events to get most recent edit
events_df = events_df.sort_values('createdAt', ascending=False).drop_duplicates(subset=['document_id'], keep='first')

In [4]:
# Merge events with userinvites on event ID
events_w_invites = events_df.merge(userinvites_df, left_on='document_id', right_on='event_id', how='left', suffixes=('_event', '_invite'))
#df num events created per day that have invites
unique_events_per_day = (
    events_w_invites
    .dropna(subset=['document_id_invite'])
    .groupby(pd.Grouper(key='createdAt_event', freq='D'))['document_id_event']
    .nunique()
    .reset_index(name='unique_event_count')
)

In [5]:
#Graph number of events created per day with invites
fig = px.bar(unique_events_per_day, x='createdAt_event', y='unique_event_count', title='Number of Events Created Per Day with Invites')
fig.show()

## Attendees Per Event

In [6]:
count_rsvp = events_w_invites[events_w_invites['status'] == 'accepted'].groupby('document_id_event').size().reset_index(name='accepted_invite_count')

In [7]:
#merge with events to get event details
count_rsvp = count_rsvp.merge(events_df, left_on='document_id_event', right_on='document_id', how='left')

In [8]:
count_rsvp['createdAt_date'] = count_rsvp['createdAt'].dt.date
px.bar(count_rsvp[count_rsvp['type'] != 'checkin'], x='createdAt_date', y='accepted_invite_count', color='type', title='Number of Accepted Invites per Event by Type').show()

In [31]:
percent_invite_by_type = count_rsvp.groupby('type')['accepted_invite_count'].sum().reset_index(name='sum_accepted_invites')
percent_invite_by_type = percent_invite_by_type[percent_invite_by_type['type'] != 'checkin']

In [32]:
percent_invite_by_type['percentage'] = (percent_invite_by_type['sum_accepted_invites'] / percent_invite_by_type['sum_accepted_invites'].sum()) * 100

In [33]:
fig = px.pie(percent_invite_by_type, names='type', values='percentage', width=550, height=400)
fig.update_layout(
    title={
        'text': "% of Accepted Invites by Event Type",
        'y': 0.92,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

In [38]:
import pytz

# Make the comparison timestamp timezone-aware (UTC)
#average acceptance per event current period and previous period 
now_utc = pd.Timestamp.now(tz='UTC')
two_weeks_ago = now_utc - pd.Timedelta(weeks=2)
four_weeks_ago = now_utc - pd.Timedelta(weeks=4)

avg_acceptance_current = count_rsvp[count_rsvp['createdAt'] >= two_weeks_ago]['accepted_invite_count'].mean()
avg_acceptance_previous = count_rsvp[(count_rsvp['createdAt'] < two_weeks_ago) & (count_rsvp['createdAt'] >= four_weeks_ago)]['accepted_invite_count'].mean()
import plotly.graph_objects as go

# Get the accepted invite counts for the two periods
recent_value = avg_acceptance_current if avg_acceptance_current is not None else 0
prior_value = avg_acceptance_previous if avg_acceptance_previous is not None else 0

fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = recent_value,
    delta = {'reference': prior_value, 'relative': False},
    title = {'text': "Average Accepted Invites per Event (Last 2 Weeks)"},
    gauge = {'shape': "bullet"}
))

fig.update_layout(
    grid = {'rows': 1, 'columns': 1, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Average Accepted Invites per Event"},
        'mode' : "number+delta",
        'delta' : {'reference': prior_value}}]
    }}
)

fig.show()

## Time to First Event

In [18]:
#on hold until I can get createdAt to be used

## Event Velocity

In [9]:
# Group by date and count events
events_per_day = events_df.groupby(events_df['createdAt'].dt.date).size().reset_index(name='events_created')


# Calculate day-over-day velocity
events_per_day['velocity'] = events_per_day['events_created'].diff()

In [10]:
# Filter to last 90 days
days_90 = (pd.Timestamp.now() - pd.Timedelta(days=90)).date()
events_per_day = events_per_day[events_per_day['createdAt'] >= days_90]

In [30]:
#plotting event velocity 
fig = px.line(events_per_day, x='createdAt', y='velocity',
               title='Event Creation Velocity (Last 90 Days)',
               width=1000, height=400)
#adding rectangle over last 2 weeks to highlight
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=(pd.Timestamp.now() - pd.Timedelta(weeks=2)).date(),
    y0=0,
    x1=pd.Timestamp.now().date(),
    y1=1,
    fillcolor="LightSalmon",
    opacity=0.5,
    layer="below",
    line_width=0,
)
#adding annotation for the highlighted area
fig.add_annotation(
    x=(pd.Timestamp.now() - pd.Timedelta(weeks=1)).date(),
    y=1.05,
    xref="x",
    yref="paper",
    text="Last 2 Weeks",
    showarrow=False,
    font=dict(size=12, color="Black")
)